# Исследование надежности заемщиков.

Описание проекта  

Заказчик — кредитный отдел банка. 

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.   

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
# загрузим таблицу и посмотрим на первые 10 строк таблицы

borrower_reliability = pd.read_csv('data.csv')
borrower_reliability.head(10)




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В глаза сразу бросаются отрицательные значения в столбце -***days_employed***, разная высота шрифта - ***education***,
большое количество цифр - ***total_income***


### Шаг 2.1 Заполнение пропусков


In [3]:
# проверим кол-во пропусков

borrower_reliability.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**days_employed** и **total_income** 2174/21525= 10% не заполнены

Проверим сколько значений в столбце **'days_employed'** >0 и сколько <0:

In [4]:
print("Количество строк с 'days_employed' > 0:", borrower_reliability[borrower_reliability['days_employed'] > 0].shape[0])
print("Количество строк с 'days_employed' < 0:", borrower_reliability[borrower_reliability['days_employed'] < 0].shape[0])

Количество строк с 'days_employed' > 0: 3445
Количество строк с 'days_employed' < 0: 15906


Найдем медиану всех значенией **"total_income"**, сгруппированных по типу **"income_type"** и найдя медианные значения **"total_income"** заполним пустые значения для каждого типа.

In [5]:
med_total_income = borrower_reliability.groupby('income_type')['total_income'].median()


print(med_total_income)



income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


Проверим количество пропусков по каждому типу **'income_type'**:

In [6]:
print('Кол-во пропусков в безработный:', borrower_reliability.query('income_type=="безработный" ')['total_income'].isna().sum())
print('Кол-во пропусков в в декрете:', borrower_reliability.query('income_type=="в декрете" ')['total_income'].isna().sum())
print('Кол-во пропусков в госслужащий:', borrower_reliability.query('income_type=="госслужащий" ')['total_income'].isna().sum())
print('Кол-во пропусков в компаньон:', borrower_reliability.query('income_type=="компаньон" ')['total_income'].isna().sum())
print('Кол-во пропусков в пенсионер:', borrower_reliability.query('income_type=="пенсионер" ')['total_income'].isna().sum())
print('Кол-во пропусков в предприниматель:', borrower_reliability.query('income_type=="предприниматель" ')['total_income'].isna().sum())
print('Кол-во пропусков в сотрудник:', borrower_reliability.query('income_type=="сотрудник" ')['total_income'].isna().sum())
print('Кол-во пропусков в студент:', borrower_reliability.query('income_type=="студент" ')['total_income'].isna().sum())

Кол-во пропусков в безработный: 0
Кол-во пропусков в в декрете: 0
Кол-во пропусков в госслужащий: 147
Кол-во пропусков в компаньон: 508
Кол-во пропусков в пенсионер: 413
Кол-во пропусков в предприниматель: 1
Кол-во пропусков в сотрудник: 1105
Кол-во пропусков в студент: 0


Заменим пустые значения в **госслужащий-компаньон-пенсионер-предприниматель-сотрудник** соответствующими значениями из **med_total_income**:

In [7]:
borrower_reliability.loc[(borrower_reliability['total_income'].isnull())\
                          & (borrower_reliability['income_type'] == "госслужащий"),\
                          'total_income'] = med_total_income['госслужащий']
print('Кол-во пропусков в госслужащий:', borrower_reliability.query('income_type=="госслужащий" ')['total_income'].isna().sum())

borrower_reliability.loc[(borrower_reliability['total_income'].isnull())\
                          & (borrower_reliability['income_type'] == "компаньон"),\
                          'total_income'] = med_total_income['компаньон']
print('Кол-во пропусков в компаньон:', borrower_reliability.query('income_type=="компаньон" ')['total_income'].isna().sum())

borrower_reliability.loc[(borrower_reliability['total_income'].isnull())\
                          & (borrower_reliability['income_type'] == "пенсионер"),\
                          'total_income'] = med_total_income['пенсионер']
print('Кол-во пропусков в пенсионер:', borrower_reliability.query('income_type=="пенсионер" ')['total_income'].isna().sum())

borrower_reliability.loc[(borrower_reliability['total_income'].isnull())\
                          & (borrower_reliability['income_type'] == "предприниматель"),\
                          'total_income'] = med_total_income['предприниматель']
print('Кол-во пропусков в предприниматель:', borrower_reliability.query('income_type=="предприниматель" ')['total_income'].isna().sum())

borrower_reliability.loc[(borrower_reliability['total_income'].isnull())\
                          & (borrower_reliability['income_type'] == "сотрудник"),\
                          'total_income'] = med_total_income['сотрудник']
print('Кол-во пропусков в сотрудник:', borrower_reliability.query('income_type=="сотрудник" ')['total_income'].isna().sum())


Кол-во пропусков в госслужащий: 0
Кол-во пропусков в компаньон: 0
Кол-во пропусков в пенсионер: 0
Кол-во пропусков в предприниматель: 0
Кол-во пропусков в сотрудник: 0


In [8]:
borrower_reliability['total_income'].isna().sum()

0

In [9]:
borrower_reliability.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Столбец **'children'**: 


In [10]:
borrower_reliability['children'].value_counts() #количество уникальных совпадений в 'children' 


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

**Аномалии в 'children':**  
- 76 значений '20'    
- 47 значений '-1'    

In [11]:
# посмотрим на строки где кол-во детей "-1"

borrower_reliability[borrower_reliability['children']==-1].head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [12]:
# посмотрим на строки где кол-во детей "20"

borrower_reliability[borrower_reliability['children']==20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,150447.935283,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,172357.950966,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


Закономерности не прослеживаются в **'children'**  
Так как количество не существенно        
Будем считать -1 == 1 и 20 == 2, что произошла опечатка данных   

In [13]:
borrower_reliability.loc[borrower_reliability['children'] == 20, 'children'] = 2
borrower_reliability.loc[borrower_reliability['children'] == -1, 'children'] = 1
borrower_reliability['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Столбец **'days_employed'**:

In [14]:
borrower_reliability['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

**Аномалии в 'days_employed'**:
    
- max значение: **401755** дней (**1100 лет**).    
- mean значение: **60378** дней (**165 лет**).      
- Будем считать что произошла ошибка данных при переносе в таблицу.    
- Так как данные не информативны удалим стобец


In [15]:
del borrower_reliability['days_employed']
borrower_reliability.head(10)


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Столбец **'education'**:

In [16]:
borrower_reliability['education'].value_counts()


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [17]:
# приведем к нижнему регистру

borrower_reliability['education'] = borrower_reliability['education'].str.lower()
borrower_reliability['education'].value_counts(), borrower_reliability['education_id'].value_counts()



(среднее                15233
 высшее                  5260
 неоконченное высшее      744
 начальное                282
 ученая степень             6
 Name: education, dtype: int64,
 1    15233
 0     5260
 2      744
 3      282
 4        6
 Name: education_id, dtype: int64)

Столбец **'dob_years'**:

In [18]:
borrower_reliability['dob_years'].value_counts().sort_index()


0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [19]:
# заменим 0 на средний возраст
borrower_reliability.loc[borrower_reliability['dob_years'] == 0, 'dob_years'] = borrower_reliability['dob_years'].mean() 
# заменим тип на Int
borrower_reliability['dob_years'] = borrower_reliability['dob_years'].astype('int')
#проверим замену
borrower_reliability['dob_years'].value_counts().sort_index()  

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Столбец **'gender'**:  

In [20]:
borrower_reliability['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

1 значение **XNA=1**, изменим его на **F**, так как его большинство!

In [21]:
borrower_reliability.loc[borrower_reliability['gender'] == 'XNA', 'gender'] = 'F'
borrower_reliability['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Столбец **'income_type'**:  

In [22]:
borrower_reliability['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

- **'income_type'** в порядоке      


In [23]:
borrower_reliability['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

- **'debt'** в порядоке      


In [24]:
borrower_reliability['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Шаг 2.3. Изменение типов данных.

In [25]:
# изменим тип данных в столбце ['total_income'] на integer

borrower_reliability['total_income'] = borrower_reliability['total_income'].astype('int')
borrower_reliability.info(), borrower_reliability.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int32 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int32 
 10  purpose           21525 non-null  object
dtypes: int32(2), int64(4), object(5)
memory usage: 1.6+ MB


(None,
    children  dob_years education  education_id     family_status  \
 0         1         42    высшее             0   женат / замужем   
 1         1         36   среднее             1   женат / замужем   
 2         0         33   среднее             1   женат / замужем   
 3         3         32   среднее             1   женат / замужем   
 4         0         53   среднее             1  гражданский брак   
 
    family_status_id gender income_type  debt  total_income  \
 0                 0      F   сотрудник     0        253875   
 1                 0      F   сотрудник     0        112080   
 2                 0      M   сотрудник     0        145885   
 3                 0      M   сотрудник     0        267628   
 4                 1      F   пенсионер     0        158616   
 
                       purpose  
 0               покупка жилья  
 1     приобретение автомобиля  
 2               покупка жилья  
 3  дополнительное образование  
 4             сыграть свадьбу  

### Шаг 2.4. Удаление дубликатов.

In [26]:
# кол-во дубликатов

borrower_reliability.duplicated().sum()

71

In [27]:
# удаленим дубликаты

borrower_reliability = borrower_reliability.drop_duplicates().reset_index(drop=True) #удаление дубликатов

In [28]:
# проверка, после удаления дубликатов

borrower_reliability.duplicated().sum() 

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:

- Каждому уникальному значению из education соответствует уникальное значение **education_id** — в первом;  
- Каждому уникальному значению из family_status соответствует уникальное значение **family_status_id** — во втором.
  

In [29]:

education_id = borrower_reliability[['education', 'education_id']] #создаем словарь education_id

education_id = education_id .drop_duplicates().reset_index(drop=True) # удаляем дубликаты из словаря

education_id


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [30]:
family_status_id = borrower_reliability[['family_status', 'family_status_id']] #создаем словарь family_status_id

family_status_id = family_status_id.drop_duplicates().reset_index(drop=True) # удаляем дубликаты из словаря


family_status_id

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим столбы **['education']** и **['family_status']** и проверим

In [31]:
del borrower_reliability['education'] #удаляем стобец 'education'
del borrower_reliability['family_status'] #удаляем столбец 'family_status'


borrower_reliability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   dob_years         21454 non-null  int32 
 2   education_id      21454 non-null  int64 
 3   family_status_id  21454 non-null  int64 
 4   gender            21454 non-null  object
 5   income_type       21454 non-null  object
 6   debt              21454 non-null  int64 
 7   total_income      21454 non-null  int32 
 8   purpose           21454 non-null  object
dtypes: int32(2), int64(4), object(3)
memory usage: 1.3+ MB


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, создадим новый столбец **total_income_category**

In [32]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    elif total_income <= 50000:
        return 'D'
    elif total_income <= 200000:
        return 'C'
    elif total_income <= 1000000:
        return 'B'
    return 'A'


borrower_reliability['total_income_category'] = borrower_reliability['total_income'].apply(total_income_category)
borrower_reliability.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [33]:
borrower_reliability['total_income_category'].value_counts()

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [34]:
borrower_reliability['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [35]:
def purpose_category(i):
    if "автомобил" in i:
        return 'операции с автомобилем'
    elif 'недвиж' in i or 'жиль' in i:
        return 'операции с недвижимостью'
    elif 'свадьб' in i:
        return 'проведение свадьбы'
    else:
        return 'получение образования'
    
borrower_reliability['purpose_category']=borrower_reliability['purpose'].apply(purpose_category) 
borrower_reliability.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [36]:
borrower_reliability['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сформирует **сводную таблицу** которая покажет количество **задолженности ('debt')** по количеству детей:

In [37]:
data_pivot_children = borrower_reliability.pivot_table(index=['children'], values=['debt'], aggfunc='sum')
data_pivot_children.columns = ['Задолженность']
data_pivot_children

,Задолженность
children,
0,1063
1,445
2,202
3,27
4,4
5,0


In [38]:
def analyse_debt(column_list):
    analyse_table = borrower_reliability.groupby(column_list).agg({'debt':['sum', 'count']})
    analyse_table[('debt', '% не выплат')] = analyse_table[('debt', 'sum')] / analyse_table[('debt', 'count')]*100
    return analyse_table

analyse_debt(['children'])

debt                   
           sum  count % не выплат
children                         
0         1063  14091    7.543822
1          445   4855    9.165808
2          202   2128    9.492481
3           27    330    8.181818
4            4     41    9.756098
5            0      9    0.000000

##### Вывод 1:
Из полученных данных видно, что больше всего берут кредиты те у кого нет детей, количество 
не выплат у них тоже выше остальных. Но процент не выплат у данной категории самый низкий на уровне 7,5%
   
Далее идут люди у которых 1 ребенок, процент не выплат у них уже выше и достигает 9,15%

У кого 2 ребенка высокий процент не выплаты кредита - 9,48% (почти как и в ситуации с 4 детьми), хотя по поддержке государства за второго ребенка семьи получают материнский капиталл.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
data_pivot_children = borrower_reliability.pivot_table(index=['family_status_id'], values=['debt'], aggfunc='sum')
analyse_debt(['family_status_id'])

debt                   
                  sum  count % не выплат
family_status_id                        
0                 931  12339    7.545182
1                 388   4151    9.347145
2                  63    959    6.569343
3                  85   1195    7.112971
4                 274   2810    9.750890

##### Вывод 2:
Из полученных данных видно, что больше всего берут кредиты **0-женатые/замужние**, количество 
не выплат у них тоже выше остальных. Но процент не выплат у данной категории низкий на уровне 7,5%
Ниже процент не выплат у **3-в разводе** и равен 7,1%

Самый низкий процент не выплат имеют **2-вдовец/вдова** - на уровне 6,6%
   
Категории **1-гражданский брак** и **4-не женат/не замужем** имеют процент не выплат в районе 9.3-9.8%

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:

analyse_debt(['total_income_category'])

debt                   
                        sum  count % не выплат
total_income_category                         
A                         2     25    8.000000
B                       356   5042    7.060690
C                      1360  16015    8.492039
D                        21    350    6.000000
E                         2     22    9.090909

##### Вывод 3:

Самый большая часть которая берет кредиты это люди с зп **"С"(50-200 тыс)**. Она практически в 3 раза больше всех остальные вместе взятых категорий. Процент не выплат составляет при этом 8,5%

На 2 месте люди с доходом **"B"(200-1000тыс)**, доля кредитов значительно меньше категории С. Процент не выплат составляет 7,1%

Самый низкий процент не выплат имеют люди с зп **"D"(30-50тыс)** он равен 6%

Люди с доходом свыше 1млн и ниже 30тыс, особо не берут кредиты, но при этом имеют 8% и 9% соответственно


##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [41]:
analyse_debt(['purpose_category'])

debt                   
                          sum  count % не выплат
purpose_category                                
операции с автомобилем    403   4306    9.359034
операции с недвижимостью  782  10811    7.233373
получение образования     370   4013    9.220035
проведение свадьбы        186   2324    8.003442

##### Вывод 4:

Больше всего кредитов имеет категория **"операции с недвижимостью"**, но зато доля просрочек по категории 7,2% - самая низкая.

В категориях **"операции с автомобилем"** и **"получение образования"** число взятых кредитов, а также процент невыплат примерно одинаков и составляет около 9,2-9.3%.

Не выплат в более мелкой категории **"проведение свадьбы"** и доля просрочек 7,9% (всего на 0,7% выше чем **"недвижимость"**) - невысоко по сравнению **"операции с автомобилем"** и **"получение образования"**.
  

## Общий вывод:

В ходе процесса мы проанализировали данные.  

При предобработке данных было удалено около 1% некорректных строк и 1 некорректный столбец(который для поставленных задач не был необходим).

Поменяли в 1 столбце типы - для лучшей наглядности и более быстрой обработки. Удалили дубликаты. 

Для одного столбца сделали категоризацию.

После подготовки данных, сделали сводные таблицы (ипользовали функцию) и ответили на все 4 поставленных вопроса.

Были выявили закономерности в просрочках по кредитам(где-то ярко выраженные, а где-то нет).
